In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, metrics
from sklearn import svm, neighbors, neural_network
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import datetime
import pickle
import json

In [2]:
## Read training data, labels, test data
train_features_original = pd.read_csv('dataset/train_features.csv', delimiter=',')
train_labels_original   = pd.read_csv('dataset/train_labels.csv'  , delimiter=',')
test_features_original  = pd.read_csv('dataset/test_features.csv' , delimiter=',')

In [3]:
# sort train data and labels values by pid, so they are the same (because after with the manipulation of data things get nasty)
train_features_original = train_features_original.sort_values(['pid','Time'])
train_labels_original   = train_labels_original.sort_values(['pid'])
test_features_original  = test_features_original.sort_values(['pid','Time'])

# Use pid as index
train_labels_original.set_index("pid", inplace=True)

In [4]:
train_features_original

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172879,31658,8,60.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,96.0,NaN,NaN,NaN,71.0,NaN,NaN,127.0,NaN
172880,31658,9,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172881,31658,10,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172882,31658,11,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.0,NaN,NaN,NaN,71.0,NaN,NaN,135.0,NaN


In [5]:
train_labels_original

,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
pid,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20.4,99.1,95.4,65.8
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,17.8,78.8,97.4,71.8
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,17.9,75.1,97.3,80.7
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.7,112.8,97.0,92.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.5,97.0,95.3,101.4
31654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.2,119.2,97.6,91.8
31656,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,21.0,93.8,99.2,92.2


# Some preprocessing

If the series of 12 measurements has at least 2 non NaN:
* replace the series of 12 values by their average and the slope of a 1D fit

If the series of 12 measurements has exactly 1 non NaN:
* replace the series of 12 values by their average
* a value that should be imputed, see (1)

If the series of 12 measurements has only NaNs:
* a value that should be imputed, see (2)
* a value that should be imputed, see (1)

(1) Replace the value to be imputed by:
* either the average of the slopes that can be computed
* or 0.
* or -9999.

(2) Replace the value to be imputed by:
* either the average of the average that can be computed
* or 0.
* or -9999.

In [6]:
# Make list of PIDs
pids = train_features_original["pid"].drop_duplicates().to_list()
Npatients_original = len(pids)
print("Number of patients: %d" %Npatients_original)
print("Dataset length: %d" %(len(train_features_original)))

Number of patients: 18995
Dataset length: 227940


In [49]:
# Reducing training for speeding up tests
Npatients_lite = Npatients_original//1000
pids_lite = pids[:Npatients_lite]

train_features_lite = train_features_original[train_features_original["pid"].isin(pids_lite)]
print("Lite number of patients: %d" %Npatients_lite)
print("Lite dataset length: %d" %(len(train_features_lite)))

train_labels_lite = train_labels_original[train_labels_original.index.isin(pids_lite)]

Lite number of patients: 18
Lite dataset length: 216


In [50]:
# Decide here if to use the lite dataset or the whole dataset
train_features = train_features_lite.copy()
train_labels   = train_labels_lite.copy()
pids = pids_lite.copy()
Npatients = Npatients_lite

In [51]:
# Make list of feature names
feature_names = [ x for x in train_features.columns  if x not in ("pid", "Time") ]
print("Features:")
print(feature_names)

Features:
['Age', 'EtCO2', 'PTT', 'BUN', 'Lactate', 'Temp', 'Hgb', 'HCO3', 'BaseExcess', 'RRate', 'Fibrinogen', 'Phosphate', 'WBC', 'Creatinine', 'PaCO2', 'AST', 'FiO2', 'Platelets', 'SaO2', 'Glucose', 'ABPm', 'Magnesium', 'Potassium', 'ABPd', 'Calcium', 'Alkalinephos', 'SpO2', 'Bilirubin_direct', 'Chloride', 'Hct', 'Heartrate', 'Bilirubin_total', 'TroponinI', 'ABPs', 'pH']


In [52]:
# Replace the series of 12 measurements by their average and slope
# This takes a lot of time...
print(datetime.datetime.now())

def make_linear_fit(x, y):
    if len(x) < 2:
        return np.nan
    else:
        return np.polyfit(x, y, 1)[0]


train_dict = {}
pid_filters = [ (train_features["pid"] == pid) for pid in pids ]
for feature_name in feature_names:
    if feature_name == "Age":
        train_dict[feature_name + "_n"] = [12] * Npatients
        train_dict[feature_name + "_avg"] = [ np.mean(train_features["Age"][train_features["pid"] == pid]) for pid in pids ]
        train_dict[feature_name + "_slope"] = [0.] * Npatients
    else:
        train_dict[feature_name] = [ train_features[feature_name][pid_filter] for pid_filter in pid_filters ]
        train_dict[feature_name + "_n"] = [ len(x.dropna()) for x in train_dict[feature_name] ]
        train_dict[feature_name + "_avg"] = [ np.mean(x) for x in train_dict[feature_name] ]
        train_dict[feature_name + "_slope"] = [ make_linear_fit(train_features["Time"].where((pid_filter) & (train_features[feature_name].notna())).dropna(), x.dropna()) for x, pid_filter in zip(train_dict[feature_name], pid_filters) ]
    print("%s finished" %feature_name)
    
print(datetime.datetime.now())

2021-04-21 11:47:06.746598
Age finished
EtCO2 finished
PTT finished
BUN finished
Lactate finished
Temp finished
Hgb finished
HCO3 finished
BaseExcess finished
RRate finished
Fibrinogen finished
Phosphate finished
WBC finished
Creatinine finished
PaCO2 finished
AST finished
FiO2 finished
Platelets finished
SaO2 finished
Glucose finished
ABPm finished
Magnesium finished
Potassium finished
ABPd finished
Calcium finished
Alkalinephos finished
SpO2 finished
Bilirubin_direct finished
Chloride finished
Hct finished
Heartrate finished
Bilirubin_total finished
TroponinI finished
ABPs finished
pH finished
2021-04-21 11:47:14.133746


In [53]:
# Saving pre-processed data
jsondata = train_dict.copy()
for feature_name in feature_names:
    try:
        jsondata.pop(feature_name)
    except:
        print(feature_name)
with open('train_dict.txt', 'w') as outfile:
    json.dump(jsondata, outfile)

Age


In [26]:
train_dict["EtCO2_slope"]

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 -3.2428571428571438,
 nan,
 -0.9655647382920121,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 4.999999999999998,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,


In [27]:
# Replace NaNs of a column by the average of the column
feature_averages = {}
for feature_name in feature_names:
    train_dict[feature_name + "_avg_noNaN"]   = [ x if not np.isnan(x) else 0. for x in train_dict[feature_name + "_avg"] ]
    train_dict[feature_name + "_slope_noNaN"] = [ x if not np.isnan(x) else 0. for x in train_dict[feature_name + "_slope"] ]
    feature_averages[feature_name + "_avg"] = np.average(train_dict[feature_name + "_avg_noNaN"], weights=train_dict[feature_name + "_n"])
    feature_averages[feature_name + "_slope"] = np.average(train_dict[feature_name + "_slope_noNaN"], weights=train_dict[feature_name + "_n"])

In [28]:
feature_averages

{'Age_avg': 62.60084254870985,
 'Age_slope': 0.0,
 'EtCO2_avg': 32.419786096256686,
 'EtCO2_slope': -0.008487797828291984,
 'PTT_avg': 40.182739861523245,
 'PTT_slope': -0.6362150150443225,
 'BUN_avg': 23.120751802265705,
 'BUN_slope': -0.01571822371754862,
 'Lactate_avg': 2.835894632206759,
 'Lactate_slope': -0.08459477711722159,
 'Temp_avg': 36.84988536261615,
 'Temp_slope': 0.06529203096426663,
 'Hgb_avg': 10.567268112598061,
 'Hgb_slope': -0.007896678292557656,
 'HCO3_avg': 23.53391959798995,
 'HCO3_slope': 0.036863731870858306,
 'BaseExcess_avg': -1.0051269288203084,
 'BaseExcess_slope': 0.021879101324412207,
 'RRate_avg': 18.13236468146256,
 'RRate_slope': 0.05766959548540522,
 'Fibrinogen_avg': 278.352422907489,
 'Fibrinogen_slope': 3.3197161987390897,
 'Phosphate_avg': 3.5896883593033917,
 'Phosphate_slope': -0.002722312893876531,
 'WBC_avg': 12.155412995594716,
 'WBC_slope': -0.05176830790257242,
 'Creatinine_avg': 1.4935263462651995,
 'Creatinine_slope': 0.0007187143551233848

In [29]:
# Make features to use in training
train_features_preprocessed = pd.DataFrame()

# Add pids
train_features_preprocessed["pid"] = pids
train_features_preprocessed.set_index("pid", inplace=True)

def std_scaler(array):
    mean = np.mean(array)
    std = np.std(array, ddof=1)
    if std != 0:
        return (array-mean)/std
    else:
        return array

# Add features
for feature_name in feature_names:
    train_features_preprocessed[feature_name + "_avg"] = train_dict[feature_name + "_avg"]
    train_features_preprocessed[feature_name + "_avg"].replace(np.nan, feature_averages[feature_name + "_avg"], inplace=True)
    # Std scaling
    train_features_preprocessed[feature_name + "_avg"] = std_scaler(train_features_preprocessed[feature_name + "_avg"])

    if feature_name != "Age":
        train_features_preprocessed[feature_name + "_n"] = train_dict[feature_name + "_n"]
        train_features_preprocessed[feature_name + "_slope"] = train_dict[feature_name + "_slope"]
        train_features_preprocessed[feature_name + "_slope"].replace(np.nan, feature_averages[feature_name + "_slope"], inplace=True)
        # Std scaling
        train_features_preprocessed[feature_name + "_slope"] = std_scaler(train_features_preprocessed[feature_name + "_slope"])
    
print(train_features_preprocessed.head())
print(len(train_features_preprocessed))

      Age_avg  EtCO2_avg  EtCO2_n  EtCO2_slope   PTT_avg  PTT_n  PTT_slope  \
pid                                                                          
1   -1.708968  -0.012238        0     0.008637  0.041580      0   0.039039   
2    1.398155  -0.012238        0     0.008637 -0.494195      1   0.039039   
4    0.203108  -0.012238        0     0.008637 -0.315235      1   0.039039   
6    0.203108  -0.012238        0     0.008637  0.911914      1   0.039039   
8   -1.230949  -0.012238        0     0.008637  0.041580      0   0.039039   

      BUN_avg  BUN_n  BUN_slope  ...  Bilirubin_total_slope  TroponinI_avg  \
pid                              ...                                         
1   -0.675449      3   0.039784  ...                0.00382       0.007644   
2    0.581357      1  -0.000523  ...                0.00382      -0.706173   
4   -0.926810      2   0.039784  ...                0.00382      -0.733844   
6    0.581357      2   0.039784  ...                0.00382    

# Subtask 3

## Training

In [30]:
# Make target variables dataframe
col_numbers = [0, 11, 12, 13, 14]
train_labels_3 = train_labels.iloc[:, col_numbers]

# Restrict to pid existing in features dataset
train_labels_3 = train_labels_3[train_labels_3.index.isin(pids)]

print(train_labels_3.head())
print(len(train_labels_3))

     LABEL_BaseExcess  LABEL_RRate  LABEL_ABPm  LABEL_SpO2  LABEL_Heartrate
pid                                                                        
1                 1.0         12.1        85.4       100.0             59.9
2                 0.0         20.4        99.1        95.4             65.8
4                 0.0         17.8        78.8        97.4             71.8
6                 1.0         17.9        75.1        97.3             80.7
8                 0.0         18.7       112.8        97.0             92.6
1899


In [31]:
# Select a subset of training features
#features_names_used = [ x for x in train_features_preprocessed.columns if x.endswith("_avg") ]
features_names_used = [ x for x in train_features_preprocessed.columns if (x.endswith("_avg") or x.endswith("_slope")) ]
#features_names_used = train_features_preprocessed.columns

print("Used features:")
print(features_names_used)
print("Number of used features: %d" %(len(features_names_used)))

Used features:
['Age_avg', 'EtCO2_avg', 'EtCO2_slope', 'PTT_avg', 'PTT_slope', 'BUN_avg', 'BUN_slope', 'Lactate_avg', 'Lactate_slope', 'Temp_avg', 'Temp_slope', 'Hgb_avg', 'Hgb_slope', 'HCO3_avg', 'HCO3_slope', 'BaseExcess_avg', 'BaseExcess_slope', 'RRate_avg', 'RRate_slope', 'Fibrinogen_avg', 'Fibrinogen_slope', 'Phosphate_avg', 'Phosphate_slope', 'WBC_avg', 'WBC_slope', 'Creatinine_avg', 'Creatinine_slope', 'PaCO2_avg', 'PaCO2_slope', 'AST_avg', 'AST_slope', 'FiO2_avg', 'FiO2_slope', 'Platelets_avg', 'Platelets_slope', 'SaO2_avg', 'SaO2_slope', 'Glucose_avg', 'Glucose_slope', 'ABPm_avg', 'ABPm_slope', 'Magnesium_avg', 'Magnesium_slope', 'Potassium_avg', 'Potassium_slope', 'ABPd_avg', 'ABPd_slope', 'Calcium_avg', 'Calcium_slope', 'Alkalinephos_avg', 'Alkalinephos_slope', 'SpO2_avg', 'SpO2_slope', 'Bilirubin_direct_avg', 'Bilirubin_direct_slope', 'Chloride_avg', 'Chloride_slope', 'Hct_avg', 'Hct_slope', 'Heartrate_avg', 'Heartrate_slope', 'Bilirubin_total_avg', 'Bilirubin_total_slope',

In [32]:
# split train data into train and validation
X_train, X_test, y_train, y_test = train_test_split(train_features_preprocessed[features_names_used], train_labels_3, train_size=0.8)

In [33]:
X_train

,Age_avg,EtCO2_avg,EtCO2_slope,PTT_avg,PTT_slope,BUN_avg,BUN_slope,Lactate_avg,Lactate_slope,Temp_avg,...,Heartrate_avg,Heartrate_slope,Bilirubin_total_avg,Bilirubin_total_slope,TroponinI_avg,TroponinI_slope,ABPs_avg,ABPs_slope,pH_avg,pH_slope
pid,,,,,,,,,,,,,,,,,,,,,
2918,0.561622,-0.012238,0.008637,-0.912831,0.351683,0.707038,-10.950094,0.145337,0.017309,0.328697,...,1.818860,-1.145997,0.027601,0.00382,0.007644,-0.003337,2.078712,-0.927491,-0.072687,-0.014839
2778,-0.035902,-0.012238,0.008637,0.041580,0.039039,0.023383,-0.000523,0.145337,0.017309,-0.181292,...,-1.311846,0.484214,0.027601,0.00382,0.007644,-0.003337,3.257508,-2.442168,-0.072687,-0.014839
3125,-0.812682,-0.012238,0.008637,0.041580,0.039039,-0.738289,-0.000523,0.145337,0.017309,-0.436287,...,0.800190,-0.457169,0.027601,0.00382,0.007644,-0.003337,-0.244477,-1.992717,-0.072687,-0.014839
1402,-1.947977,-0.012238,0.008637,0.041580,0.039039,0.023383,-0.000523,0.145337,0.017309,-0.181292,...,0.329604,-0.585011,0.027601,0.00382,0.007644,-0.003337,0.657118,0.011590,-0.072687,-0.014839
153,-0.095654,-0.012238,0.008637,0.041580,0.039039,-0.465981,-0.202786,0.327475,-3.864661,0.966184,...,-0.317452,-0.752636,0.027601,0.00382,0.007644,-0.003337,-1.240186,-0.228242,-0.802704,2.434721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2697,0.800631,-0.012238,0.008637,0.041580,0.039039,-0.518348,-0.245139,0.145337,0.017309,0.711189,...,2.161529,0.112309,-0.935088,0.00382,-0.732856,-0.122040,1.107916,-0.676758,-0.072687,-0.014839
3002,-0.693178,3.146879,-1.123742,0.041580,0.039039,0.141475,-0.000523,-0.249665,0.017309,0.838687,...,-0.370673,0.861929,-0.868408,0.00382,0.007644,-0.003337,-0.410561,-0.348781,-1.325993,-0.014839
1740,-1.111444,-0.012238,0.008637,-0.404715,0.039039,-0.675449,-0.000523,0.145337,0.017309,0.838687,...,2.536878,1.361675,-0.801728,0.00382,0.007644,-0.003337,-1.041677,-0.907557,-0.072687,-0.014839


In [34]:
y_train

,LABEL_BaseExcess,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
pid,,,,,
2918,0.0,22.2,116.7,96.7,100.6
2778,0.0,16.0,106.0,99.1,76.6
3125,0.0,13.7,65.1,97.4,99.9
1402,0.0,13.2,88.8,94.1,84.5
153,1.0,22.6,73.3,96.2,95.5
...,...,...,...,...,...
2697,0.0,19.3,97.6,98.3,87.5
3002,0.0,14.9,84.1,97.8,96.7
1740,0.0,17.3,65.4,96.2,114.5


In [20]:
## Regressor
#reg = svm.SVR()
#reg = svm.SVR(kernel='rbf')
#reg = svm.SVR(kernel='poly')
#reg = svm.SVR(kernel='sigmoid')

#reg = neighbors.KNeighborsRegressor(n_neighbors=25, weights="distance")
#reg = neighbors.KNeighborsRegressor(n_neighbors=25, weights="uniform")
# whether using distance or uniform weight does not matter much

#reg = Ridge(alpha = 1, fit_intercept=True)
reg = Lasso(alpha = 0.1, fit_intercept=True)

# Multi Layer Perceptron
# Use L2 penalty (alpha)
# reg = neural_network.MLPRegressor(hidden_layer_sizes=(100),
#                                   alpha=100,        # L2 regularization
#                                   activation="relu",
#                                   solver="adam",
#                                   learning_rate_init=0.01,
#                                   learning_rate="constant",
#                                   max_iter=500)

In [21]:
# Fit to the training data
# For 1 column for now
col = 'LABEL_RRate'
#col = 'LABEL_ABPm'
#col = 'LABEL_Heartrate'

print(datetime.datetime.now())
reg.fit(X_train, y_train[col])
print(datetime.datetime.now())

2021-04-21 11:31:50.661134
2021-04-21 11:31:50.693913


In [22]:
# Prediction to evaluate the model
y_pred = reg.predict(X_test)
print("R2 score: %.2f" %(metrics.r2_score(y_test[col], y_pred)))

R2 score: 0.46


### Summary of tests

#### Avg variables only

**LABEL_RRate**     
SVR        : R2 = 0.31     
SVR rbf    : R2 = 0.31     
SVR poly   : R2 = -0.32    
SVR sigmoid: R2 = -0.44     
knn 25     : R2 = 0.20     
Ridge 0.1  : R2 = 0.38      
Ridge 100  : R2 = 0.39    
Lasso 0.1  : R2 = 0.41
MLP   hidden_layer_sizes=(40, 40),   R2 = 0.41  (but fluctuates!)      
      alpha=10,      
      activation="relu",     
      solver="adam",    
      learning_rate_init=0.02,    
      learning_rate="constant",    
      max_iter=500    

MPL   hidden_layer_sizes=(100),    R2 = 0.39        
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

**LABEL_ABPm**     
SVR        : R2 = 0.46     
SVR rbf    : R2 = 0.46     
SVR poly   : R2 = 0.13     
SVR sigmoid: R2 = 0.56     
knn 25     : R2 = 0.45          
Ridge 0.1  : R2 = 0.59     
Ridge 100  : R2 = 0.59    
Lasso 0.1  : R2 = 0.60

MPL   hidden_layer_sizes=(100),    R2 = 0.62        
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    
      
MPL   hidden_layer_sizes=(50, 50, 50, 50),   R2 = 0.60    
      alpha=300,    
      
MPL   hidden_layer_sizes=(200),   R2 = 0.60    
      alpha=200,    

MPL   hidden_layer_sizes=(100),    R2 = 0.62 (relu), 0.54 (tanh), 0.55 (logistic)    
      alpha=100,    
      activation=varied,    
      solver="adam",    
      learning_rate_init=0.01,    
      learning_rate="constant",    
      max_iter=500)   

   => Relu is the best


**LABEL_Heartrate**     
SVR        : R2 = 0.44     
SVR rbf    : R2 = 0.44    
SVR poly   : R2 = 0.19     
SVR sigmoid: R2 = 0.52     
knn 25     : R2 = 0.36     
Ridge 0.1  : R2 = 0.55    
Ridge 100  : R2 = 0.55    
Lasso 0.1  : R2 = 0.56

MPL   hidden_layer_sizes=(100),    R2 = 0.67     
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

**Conclusions**
* SVR poly looks bad
* SVR sigmoid sometimes does not work well
* SVR: no difference by linear and gaussian kernel
* Ridge works quite well for a wide range of alpha values
* Lasso works slightly better than Ridge for small alpha (<1)
  => Some variables do not bring much information
  => More complex model with L1 regularisation?
* MLP hidden_leayers=(100), alpha=100, activation="relu", learning_rate=0.01
  gives promising results


#### Avg and n variables
n variable are not std scaled

**LABEL_RRate**          
Ridge 0.1  : R2 = 0.45      
Ridge 100  : R2 = 0.46   
Lasso 0.1  : R2 = 0.45    
MPL   hidden_layer_sizes=(100),    R2 = 0.40        
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

**LABEL_ABPm**              
Ridge 0.1  : R2 = 0.62    
Ridge 100  : R2 = 0.62     
Lasso 0.1  : R2 = 0.63

MPL   hidden_layer_sizes=(100),    R2 = 0.60         
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

**LABEL_Heartrate**        
Ridge 0.1  : R2 = 0.53    
Ridge 100  : R2 = 0.54    
Lasso 0.1  : R2 = 0.54

MPL   hidden_layer_sizes=(100),    R2 = 0.50     
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

**Conclusions**
* Adding the number of measurements does not help for heartrate but seems to help a bit the others
* Will not use them 


#### Avg and slope variables

**LABEL_RRate**          
Ridge 0.1  : R2 =      
Ridge 100  : R2 =    
Lasso 0.1  : R2 =     
MPL   hidden_layer_sizes=(100),    R2 =         
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

**LABEL_ABPm**              
Ridge 0.1  : R2 =     
Ridge 100  : R2 =      
Lasso 0.1  : R2 = 

MPL   hidden_layer_sizes=(100),    R2 =          
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

**LABEL_Heartrate**        
Ridge 0.1  : R2 =     
Ridge 100  : R2 =     
Lasso 0.1  : R2 = 

MPL   hidden_layer_sizes=(100),    R2 =      
      alpha=100,    
      activation="relu",    
      solver="adam",     
      learning_rate_init=0.01,    
      learning_rate="constant",     
      max_iter=500)    

In [23]:
#save model into file
filename = '3_'+col
pickle.dump(reg, open(filename, 'wb'))